In [1]:
import pandas as pd
import numpy as np

In [2]:
medicines = pd.read_csv('./datasets Uncleaned (CSV)/medicines.csv')
inventory = pd.read_csv('./datasets Uncleaned (CSV)/inventory.csv')
facilities = pd.read_csv('./datasets Uncleaned (CSV)/facilities.csv')
prescription = pd.read_csv('./datasets Uncleaned (CSV)/prescription_demand.csv')
suppliers = pd.read_csv('./datasets Uncleaned (CSV)/suppliers.csv')
orders = pd.read_csv('./datasets Uncleaned (CSV)/supply_orders.csv')

# Medicines

In [42]:
medicines.head()

,medicine_id,name,category,manufacturer,shelf_life,cost_price,selling_price
0,100,Amoxicillin,Blood Pressure,Merck,57,61.49,81.45
1,101,Ibuprofen,Cardiac,Bayer,60,16.07,18.55
2,102,Metformin,Respiratory,Eli Lilly,21,51.96,77.86
3,103,Aspirin,Mental Health,Sanofi,36,73.96,91.44
4,104,Losartan,Digestive,Johnson & Johnson,52,64.37,71.64


In [44]:
medicines.isna().sum()

medicine_id      0
name             0
category         0
manufacturer     0
shelf_life       0
cost_price       0
selling_price    0
dtype: int64

In [43]:
medicines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   medicine_id    50 non-null     int64  
 1   name           50 non-null     object 
 2   category       50 non-null     object 
 3   manufacturer   50 non-null     object 
 4   shelf_life     50 non-null     int64  
 5   cost_price     50 non-null     float64
 6   selling_price  50 non-null     float64
dtypes: float64(2), int64(2), object(3)
memory usage: 2.9+ KB


In [6]:
medicines['shelf_life'] = medicines['shelf_life'].fillna(0)
medicines['selling_price'] = medicines['selling_price'].fillna("NA")

In [7]:
medicines['shelf_life'] = medicines['shelf_life'].astype(int)

In [8]:
#medicines['medicine_id'].unique()

In [9]:
#medicines['name'].value_counts()

In [10]:
#medicines['name'].nunique()

In [11]:
k = int(medicines['shelf_life'].mean())

for x in medicines.index:
    if medicines.loc[x, 'shelf_life'] == 0:
        medicines.loc[x, 'shelf_life'] = k

In [12]:
#medicines['shelf_life'].value_counts()

In [13]:
sp = inventory.groupby('medicine_id')['selling_price'].unique().reset_index()

In [14]:
#print(sp)

In [15]:
for x in medicines.index:
    if medicines.loc[x, 'selling_price'] == "NA":
        y = medicines.loc[x, 'medicine_id']
        for i in sp.index:
            if sp.loc[i, 'medicine_id'] == y:
                medicines.loc[x, 'selling_price'] = sp.loc[i, 'selling_price'][0]

In [16]:
medicines['selling_price'] = medicines['selling_price'].astype(float)

# Inventory

In [45]:
inventory.head()

,inventory_id,facility_id,medicine_id,quantity_in_stock,reorder_level,last_updated,cost_price,selling_price,inventory_value,expiry_date
0,1,1,100,915,124,2025-02-14,61.49,81.45,56263.35,2025-07-15
1,2,1,101,276,81,2025-02-14,16.07,18.55,4435.32,2027-08-13
2,3,1,102,520,176,2025-02-14,51.96,77.86,27019.20,2025-10-18
3,4,1,103,219,64,2025-02-14,73.96,91.44,16197.24,2027-05-20
4,5,1,104,989,76,2025-02-14,64.37,71.64,63661.93,2026-04-14


In [18]:
inventory.drop(columns=['inventory_level'], inplace=True)

In [46]:
inventory.isna().sum()

inventory_id         0
facility_id          0
medicine_id          0
quantity_in_stock    0
reorder_level        0
last_updated         0
cost_price           0
selling_price        0
inventory_value      0
expiry_date          0
dtype: int64

In [47]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   inventory_id       500 non-null    int64         
 1   facility_id        500 non-null    int64         
 2   medicine_id        500 non-null    int64         
 3   quantity_in_stock  500 non-null    int64         
 4   reorder_level      500 non-null    int64         
 5   last_updated       500 non-null    datetime64[ns]
 6   cost_price         500 non-null    float64       
 7   selling_price      500 non-null    float64       
 8   inventory_value    500 non-null    float64       
 9   expiry_date        500 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(5)
memory usage: 39.2 KB


In [21]:
inventory['cost_price'] = inventory['cost_price'].fillna("NA")
inventory['medicine_id'] = inventory['medicine_id'].fillna("NA")

In [22]:
cp = inventory[(inventory['cost_price'] != "NA") & (inventory['medicine_id'] != "NA")].groupby('medicine_id')['cost_price'].unique().reset_index()

In [23]:
#print(cp)

In [24]:
for x in inventory.index:
    if inventory.loc[x, 'cost_price'] == "NA":
        m = inventory.loc[x, 'medicine_id']
        for i in cp.index:
           if cp.loc[i, 'medicine_id'] == m:
               inventory.loc[x, 'cost_price'] = cp.loc[i, 'cost_price'][0]

    elif inventory.loc[x, 'medicine_id'] == "NA":
        m = inventory.loc[x, 'cost_price']
        for i in cp.index:
           if cp.loc[i, 'cost_price'] == m:
               inventory.loc[x, 'medicine_id'] = cp.loc[i, 'medicine_id']

In [25]:
#inventory['facility_id'].value_counts()

In [26]:
inventory['facility_id'] = pd.to_numeric(inventory['facility_id'], errors='coerce')

In [27]:
k = 0
for i in range(len(inventory)):
    if inventory.loc[i, 'medicine_id'] == 100:
        k+=1
    elif pd.isna(inventory.loc[i, 'facility_id']):
        inventory.loc[i, 'facility_id'] = k  

In [28]:
inventory['facility_id'] = inventory['facility_id'].astype(int) 
inventory['medicine_id'] = inventory['medicine_id'].astype(int) 
inventory['last_updated'] = pd.to_datetime(inventory['last_updated'])
inventory['last_updated'] = inventory['last_updated'].dt.date
inventory['last_updated'] = pd.to_datetime(inventory['last_updated'])
inventory['cost_price'] = inventory['cost_price'].astype(float) 
inventory['expiry_date'] = pd.to_datetime(inventory['expiry_date'], errors="coerce")

# Suppliers

In [48]:
suppliers.head()

,supplier_id,name,delivery_time_avg,reliability_score
0,200,MediSupply Co.,3,0.95
1,201,Global Pharma Distributors,8,0.78
2,202,HealthBridge Suppliers,9,0.82
3,203,Evergreen Medical Supplies,5,0.90
4,204,Pioneer Drug Distributors,4,0.75


In [30]:
suppliers.drop(columns=['contact_info'], inplace=True)

# Prescription

In [14]:
prescription.head(n=30)

,prescription_id,facility_id,medicine_id,quantity_prescribed,prescription_date,selling_price,revenue,payment_mode,insurance_covered_amount,patient_paid_amount
0,1,9,148,3,2024-03-28,113.53,340.59,Insurance,237.67,102.92
1,2,6,141,2,2024-01-29,85.44,170.88,Insurance,94.18,76.70
2,3,6,114,5,2024-02-23,41.84,209.20,Cash,0.00,209.20
3,4,6,123,5,2024-03-15,32.67,163.35,Cash,0.00,163.35
4,5,5,129,2,2024-03-01,48.51,97.02,Insurance,71.62,25.40
5,6,1,140,10,2024-03-15,28.55,285.50,Credit,0.00,285.50
6,7,10,104,8,2024-01-25,71.64,573.12,Credit,0.00,573.12
8,9,5,126,1,2024-01-20,108.34,108.34,Credit,0.00,108.34
9,10,3,112,6,2024-02-22,49.87,299.22,Credit,0.00,299.22
10,11,10,129,3,2024-02-10,48.51,145.53,Cash,0.00,145.53


In [12]:
#prescription.duplicated()

In [19]:
prescription.drop_duplicates(subset = ['prescription_id'], inplace=True)

In [17]:
prescription.isna().sum()

prescription_id             0
facility_id                 0
medicine_id                 0
quantity_prescribed         0
prescription_date           0
selling_price               0
revenue                     0
payment_mode                0
insurance_covered_amount    0
patient_paid_amount         0
dtype: int64

In [7]:
prescription.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   prescription_id           2050 non-null   int64  
 1   facility_id               2050 non-null   int64  
 2   medicine_id               2050 non-null   int64  
 3   quantity_prescribed       1999 non-null   float64
 4   prescription_date         2050 non-null   object 
 5   selling_price             2050 non-null   float64
 6   revenue                   1999 non-null   float64
 7   payment_mode              2050 non-null   object 
 8   insurance_covered_amount  2037 non-null   float64
 9   patient_paid_amount       1999 non-null   float64
dtypes: float64(5), int64(3), object(2)
memory usage: 160.3+ KB


In [8]:
prescription.dropna(subset=['quantity_prescribed'], inplace=True)

In [9]:
prescription['quantity_prescribed'] = prescription['quantity_prescribed'].astype(int) 
prescription['prescription_date'] = pd.to_datetime(prescription['prescription_date'], errors='coerce')

# Supply Orders

In [3]:
orders.head()

,order_id,supplier_id,medicine_id,facility_id,quantity_ordered,order_date,expected_delivery_date,actual_delivery_date,cost_price,total_cost,delay_days
0,1,207,107,9,261,2024-03-24,2024-03-29,2024-03-31,32.88,8581.68,2
1,2,205,122,7,317,2024-03-10,2024-03-20,2024-03-20,25.17,7978.89,0
2,3,202,107,6,241,2024-02-28,2024-03-09,2024-03-11,32.88,7924.08,2
3,4,202,135,3,245,2024-03-10,2024-03-14,2024-03-13,15.15,3711.75,-1
4,5,206,142,5,409,2024-03-22,2024-03-27,2024-03-27,64.93,26556.37,0


In [10]:
orders['medicine_id'].value_counts()

medicine_id
127    29
112    27
142    26
110    24
101    24
132    24
121    24
128    24
118    23
124    23
122    23
136    22
120    22
104    22
130    22
140    22
125    21
115    21
148    21
111    21
147    21
149    21
113    21
117    20
146    20
131    20
109    20
129    20
106    20
102    19
138    19
116    19
141    19
126    19
135    18
133    18
105    18
119    18
144    17
123    17
107    16
139    16
145    16
114    16
134    16
143    16
108    16
100    15
137    14
103    10
Name: count, dtype: int64

In [5]:
orders.drop_duplicates(subset = ['order_id'], keep = 'first', inplace=True)

In [7]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                1000 non-null   int64  
 1   supplier_id             1000 non-null   int64  
 2   medicine_id             1000 non-null   int64  
 3   facility_id             1000 non-null   int64  
 4   quantity_ordered        1000 non-null   int64  
 5   order_date              1000 non-null   object 
 6   expected_delivery_date  1000 non-null   object 
 7   actual_delivery_date    1000 non-null   object 
 8   cost_price              1000 non-null   float64
 9   total_cost              1000 non-null   float64
 10  delay_days              1000 non-null   int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 93.8+ KB


In [27]:
orders.drop(columns=['delay_days'],inplace=True)

In [28]:
orders['order_date'] = pd.to_datetime(orders['order_date'], errors='coerce')
orders['expected_delivery_date'] = pd.to_datetime(orders['expected_delivery_date'], errors='coerce')
orders['actual_delivery_date'] = pd.to_datetime(orders['actual_delivery_date'], errors='coerce')

# To csv files 

In [22]:
# medicines.to_csv('medicines_cleaned.csv', index=False)
#inventory.to_csv('inventory_cleaned.csv', index=False)
# facilities.to_csv('facilities_cleaned.csv', index=False)
# suppliers.to_csv('suppliers_cleaned.csv', index=False)
#prescription.to_csv('prescription_demand_cleaned.csv', index=False)
# orders.to_csv('supply_orders_cleaned.csv', index=False)